# Chap02 - 자연어와 단어의 분산 표현

## 2.1 자연어 처리란

- 한국어와 영어 등 우리가 평소에 쓰는 말을 **자연어**(natural language)라고 한다.

- 자연어처리(**NLP**, Natural Language Processing)은 '우리의 말을 컴퓨터에게 이해시키기 위한 기술(분야)'를 의미한다.

## 2.2 시소러스(Thesaurus)

- 시소러스는 유의어 사전으로, '뜻이 같은 단어(동의어)'나 '뜻이 비슷한 단어(유의어)'가 한 그룹으로 분류되어 있다.

- 대표적인 시소러스로는 'WordNet'이 있다.

### 시소러스 문제점

- 시대 변화에 대응하기 어렵다.
    - 신조어, 단어의 의미 변화 등
    
    
- 사람을 쓰는 비용이 크다.

- 단어의 미묘한 차이를 표헌할 수 없다.

## 2.3 통계 기반 기법

- **말뭉치(corpus)** : 대량의 텍스트 데이터를 의미하며, NLP나 애플리케이션을 염두에 두고 수집된 텍스트 데이터를 말한다.

- 통계 기반 기법의 목표는 말뭉치(corpus)에서 자동으로, 그리고 효율적으로 핵심을 추출하는 것이다.

### 2.3.1 파이썬으로 말뭉치 전처리하기

In [1]:
text = 'You say goodbye and I say hello.'

text = text.lower()
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [2]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

- 단어에 ID를 부여하고, ID의 리스트로 이용할 수 있도록 해준다.

- 딕셔너리를 이용해 단어 ID와 단어를 매핑한다.

In [6]:
word_to_id = {}

for word in words:
    if word not in word_to_id:
        new_id =  len(word_to_id)
        word_to_id[word] = new_id
        
id_to_word = {id_: word for word, id_ in word_to_id.items()}

In [7]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [8]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

- 위의 딕셔너리를 사용해, 단어 ID를 검색하거나, 반대로 단어 ID를 가지고 단어를 검색할 수 있다.

In [9]:
id_to_word[1]

'say'

In [11]:
word_to_id['hello']

5

In [12]:
# 단어 목록 -> 단어 ID 목록으로 변경
import numpy as np

corpus = [word_to_id[word] for word in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [14]:
# common/util.py -> preprocess 메서드 사용
import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [16]:
corpus, word_to_id, id_to_word

(array([0, 1, 2, 3, 4, 1, 5, 6]),
 {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6},
 {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'})

### 2.3.2 단어의 분산 표현

- 단어의 분산 표현(distributional representation)은 단어를 고정 길이의 밀집 벡터(dense vector)로 표현한다.

- 밀집 벡터는 대부분의 원소가 0이 아닌 실수로 이루어진 벡터를 말한다.

### 2.3.3 분포 가설(Distributional Hypothesis)

> 단어의 의미는 주변 단어에 의해 형성 된다.

- 단어 자체에는 의미가 없고, 그 단어가 사용된 맥락(또는 문맥, context)이 해당 단어의 의미를 형성한다.

```
I drik beer. We drink wine.

I guzzle beer. We guzzle wine.
```

- 위의 예시에서 'drink'와 'guzzle'의 주변 단어(context)인 'beer, wine'을 통해, 'drink, guzzle'이 비슷한 의미를 가지는 단어라고 짐작할 수 있다.

#### Window-based Context

- 맥락(context)는 특정 단어를 중심에 둔 그 주변 단어를 말한다.

- 맥락의 크기를 window size 라고 한다.

- 일반적으로, 좌우로 똑같은 수의 단어를 맥락으로 사용하지만, 경우에 따라 왼쪽, 오른쪽만 사용할 수 있다.

![](./images/window_size.png)

### 2.3.4 동시발생 행렬(Co-occurrence Matrix)

- 주변 단어를 '세어 보는(counting)' 방법

- 특정 단어에 대해, 그 단어의 주변에 어떤 단어가 몇 번이나 등장하는지 카운팅하여 합치는 방법

In [19]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(f'corpus: {corpus}')
print(f'id_to_word: {id_to_word}')

corpus: [0 1 2 3 4 1 5 6]
id_to_word: {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


#### Co-occurrence Matrix 생성

- `window size = 1`로 설정할 경우

|      -       | you  | say  | goodbye | and  | i    | hello | .    |
| ----------- | ---- | ---- | ------- | ---- | ---- | ----- | ---- |
| **you**     | 0    | 1    | 0       | 0    | 0    | 0     | 0    |
| **say**     | 1    | 0    | 1       | 0    | 1    | 1     | 0    |
| **goodbye** | 0    | 1    | 0       | 1    | 0    | 0     | 0    |
| **and**     | 0    | 0    | 1       | 0    | 1    | 0     | 0    |
| **i**       | 0    | 1    | 0       | 1    | 0    | 0     | 0    |
| **hello**   | 0    | 1    | 0       | 0    | 0    | 0     | 1    |
| **.**       | 0    | 0    | 0       | 0    | 0    | 1     | 0    |

In [20]:
C = np.array([[0, 1, 0, 0, 0, 0, 0], 
              [1, 0, 1, 0, 1, 1, 0], 
              [0, 1, 0, 1, 0, 0, 0], 
              [0, 0, 1, 0, 1, 0, 0], 
              [0, 1, 0, 1, 0, 0, 0], 
              [0, 1, 0, 0, 0, 0, 1], 
              [0, 0, 0, 0, 0, 1, 0]], dtype=np.int32)

In [21]:
# ID가 0인 단어의 벡터 표현
print(C[0])
# ID가 4인 단어의 벡터 표현
print(C[4])

# "goodbye"의 벡터 표현
print(C[word_to_id['goodbye']])

[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
[0 1 0 1 0 0 0]


In [48]:
# common/util.py
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i  # left window_size
            right_idx = idx + i  # right window_size

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

In [49]:
window_size = 1
vocab_size = len(id_to_word)

C = create_co_matrix(corpus, vocab_size, window_size=1)
C

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]], dtype=int32)

### 2.3.5 벡터 간 유사도

- 벡터 간의 유사도를 측정하는 방법은 다양하다(e.g. 유클리드 거리, 맨하탄 거리 등)

- 단어 벡터의 유사도를 나타낼 때는 **코사인 유사도**(cosine similarity)를 자주 사용한다.

- 두 벡터 $\mathbf{x} = (x_1, x_2, \dots, x_n)$과 $\mathbf{y}=(y_1, y_2, \dots, y_n)$ 에 대하여 코사인 유사도는 다음과 같다.

$$
\text{similarity}(\mathbf{x}, \mathbf{y}) = \frac{\mathbf{x} \cdot \mathbf{y}}{\| \mathbf{x} \| \| \mathbf{y} \|} = \frac{x_1 y_1 + \cdots + x_n y_n}{\sqrt{x_{1}^{2} + \cdots + x_n^{2}} \sqrt{y_1^{2} + \cdots + y_n^{2}}}
$$

- 위 식에서 분자에는 벡터의 내적, 분모에는 각 벡터의 노름(norm)이 등장한다.

- 즉, 벡터를 정규화하고 내적을 구하는 것이라고 볼 수 있다.

- 코사인 유사도를 직관적으로 보면, '두 벡터가 가리키는 방향이 얼마나 비슷한가' 이다. 두 방향이 완전히 같다면 1, 반대면 -1 이 된다.

In [3]:
def cos_similarity(x, y, eps=1e-8):
    # epsilon 값을 추가해, 
    # 0으로 나누기 오류가 나는 것을 막아줌
    nx = x / np.sqrt(np.sum(x**2) + eps)  # x의 정규화
    ny = y / np.sqrt(np.sum(y**2) + eps)  # y의 정규화
    return np.dot(nx, ny)

In [6]:
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, cos_similarity

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]  # "you"의 단어 벡터
c1 = C[word_to_id['i']]  # 'i'의 단어 벡터
print(cos_similarity(c0, c1))

0.7071067758832467


### 2.3.6 유사 단어의 랭킹 표시

검색어와 비슷한 단어를 유사도 순으로 출력하는 함수인 `most_similar()` 구현

1. 검색어의 단어 벡터를 꺼낸다.

2. 검색어의 단어 벡터와 다른 모든 단어 벡터와의 코사인 유사도를 각각 구한다.

3. 계산한 코사인 유사도 결과를 기준으로 값이 높은 순서대로 출력한다.

In [7]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # 1) 검색어를 꺼낸다.
    if query not in word_to_id:
        print(f'{query}(을)를 찾을 수 없습니다.')
        return 
    
    print(f'\n[query] {query}')
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    # 2) 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    # 3) 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(f' {id_to_word[i]}: {similarity[i]}')
        
        count +=1
        if count >= top:
            return

In [9]:
# numpy의 artsort() 예시
# 오름차순
x = np.array([100, -20, 2])
x.argsort()

# 내림차순
(-x).argsort()

array([0, 2, 1])

In [3]:
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, most_similar

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067758832467
 i: 0.7071067758832467
 hello: 0.7071067758832467
 say: 0.0
 and: 0.0


## 2.4 통계 기반 기법 개선하기

### 2.4.1 상호정보량

